<a href="https://colab.research.google.com/github/JoseAnt96/Anexo-TFM/blob/main/Actualizaci%C3%B3n_datos_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import io
from google.colab import files
import numpy as np
import itertools
import seaborn as sns
from datetime import timedelta

In [ ]:
def upload_files ():
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')))
    return df

In [ ]:
datos = upload_files()
ranking = upload_files()
season = upload_files()

Saving datos_modelo_v2.csv to datos_modelo_v2 (5).csv
User uploaded file "datos_modelo_v2.csv" with length 1373138 bytes


Saving ranking_datos_modelos.csv to ranking_datos_modelos (4).csv
User uploaded file "ranking_datos_modelos.csv" with length 3511627 bytes


Saving resultados_2014_2018.csv to resultados_2014_2018 (3).csv
User uploaded file "resultados_2014_2018.csv" with length 1070205 bytes


In [ ]:
s14 = season[season.SEASON == 2014].reset_index().drop(["index"], axis = 1)
s15 = season[season.SEASON == 2015].reset_index().drop(["index"], axis = 1)
s16 = season[season.SEASON == 2016].reset_index().drop(["index"], axis = 1)
s17 = season[season.SEASON == 2017].reset_index().drop(["index"], axis = 1)
s18 = season[season.SEASON == 2018].reset_index().drop(["index"], axis = 1)

s14f = pd.to_datetime(s14.loc[0,'GAME_DATE_EST'])
s14i = pd.to_datetime(s14.loc[len(s14)-1,'GAME_DATE_EST'])
s15f = pd.to_datetime(s15.loc[0,'GAME_DATE_EST'])
s15i = pd.to_datetime(s15.loc[len(s15)-1,'GAME_DATE_EST'])
s16f = pd.to_datetime(s16.loc[0,'GAME_DATE_EST'])
s16i = pd.to_datetime(s16.loc[len(s16)-1,'GAME_DATE_EST'])
s17f = pd.to_datetime(s17.loc[0,'GAME_DATE_EST'])
s17i = pd.to_datetime(s17.loc[len(s17)-1,'GAME_DATE_EST'])
s18f = pd.to_datetime(s18.loc[0,'GAME_DATE_EST'])
s18i = pd.to_datetime(s18.loc[len(s18)-1,'GAME_DATE_EST'])


In [ ]:
conferencia = ranking[(ranking.STANDINGSDATE == "2019-04-10")]
conferencia = conferencia.iloc[:,[0,4]]

In [ ]:
datos = pd.merge(datos, conferencia, left_on = "HOME_TEAM_ID", right_on="TEAM_ID")
datos = datos.rename(columns={'CONFERENCE':'conferencia_home'})
datos = datos.drop(['TEAM_ID'], axis=1)
datos = pd.merge(datos, conferencia, left_on = "VISITOR_TEAM_ID", right_on="TEAM_ID")
datos = datos.rename(columns={'CONFERENCE':'conferencia_visit'})
datos = datos.drop(['TEAM_ID'], axis=1)
datos

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_WINS,ult7dias_local,ult7dias_visitante,win_ult5_local,win_ult5_visitante,numero_victorias_visitante,numero_victorias_local,mean_puntos_visitante,porcentaje_triples_visitante,porcentaje_tiros_libres_visitante,mean_rebotes_visitante,mean_asistencias_visitante,mean_robos_visitante,mean_tapones_visitante,mean_faltas_visitante,mean_player_visitane,mean_puntos_local,porcentaje_triples_local,porcentaje_tiros_libres_local,mean_rebotes_local,mean_asistencias_local,mean_robos_local,mean_tapones_local,mean_faltas_local,mean_player_local,conferencia_home,conferencia_visit
0,2014-10-04,11400001,1610612748,1610612740,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,East,West
1,2018-10-10,11800062,1610612748,1610612740,1,2.0,1.0,1.0,0.0,0.0,1.0,101.0,0.246914,0.576271,56.5,22.0,8.0,5.5,24.0,14.0,90.0,0.333333,0.608696,52.0,21.0,5.0,8.0,26.0,11.0,East,West
2,2015-02-21,21400817,1610612748,1610612740,0,1.0,1.0,2.0,1.0,32.0,27.0,92.2,0.430108,0.772727,39.2,21.0,5.8,7.4,16.4,10.0,99.8,0.428571,0.777778,41.0,23.6,9.8,3.8,17.8,10.0,East,West
3,2015-12-25,21500436,1610612748,1610612740,1,3.0,3.0,3.0,3.0,12.0,20.0,107.6,0.373984,0.838710,39.6,18.6,7.0,7.4,21.6,11.0,101.2,0.378641,0.776596,40.6,20.0,6.8,6.2,14.8,9.0,East,West
4,2017-03-15,21601007,1610612748,1610612740,1,3.0,3.0,4.0,3.0,28.0,36.0,100.0,0.340426,0.678571,45.4,22.6,8.6,4.6,16.2,10.0,107.2,0.417647,0.737374,44.2,20.8,6.6,6.2,19.4,10.0,East,West
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6584,2015-02-06,21400757,1610612759,1610612748,1,2.0,4.0,4.0,1.0,25.0,32.0,89.8,0.330000,0.673077,42.6,19.4,6.0,4.4,19.0,11.0,98.0,0.335766,0.752137,43.8,22.8,7.8,5.0,15.2,12.0,West,East
6585,2016-03-23,21501063,1610612759,1610612748,1,3.0,3.0,4.0,3.0,45.0,61.0,113.8,0.391753,0.804688,42.8,23.0,7.0,5.8,20.6,9.0,98.8,0.326531,0.829787,44.6,23.4,7.8,7.2,19.6,12.0,West,East
6586,2016-11-14,21600150,1610612759,1610612748,1,3.0,3.0,3.0,1.0,6.0,11.0,93.2,0.366667,0.700000,42.8,19.6,8.2,6.0,19.0,10.0,98.6,0.349057,0.850000,41.6,20.6,8.4,6.2,18.0,12.0,West,East
6587,2017-12-06,21700361,1610612759,1610612748,1,4.0,3.0,4.0,2.0,14.0,19.0,96.6,0.352601,0.712871,37.0,21.2,8.4,4.4,18.4,10.0,99.4,0.392308,0.803571,41.6,22.6,7.4,6.2,19.0,12.0,West,East


In [ ]:
datos = datos.sort_values(['GAME_DATE_EST'], ascending=False).reset_index()
datos = datos.drop(["index"], axis = 1)

In [ ]:
ranking['STANDINGSDATE'] = pd.to_datetime(ranking['STANDINGSDATE'])

In [ ]:
rk1 = ranking[(ranking.STANDINGSDATE >= s14i) & (ranking.STANDINGSDATE <= s14f)]  
rk2 = ranking[(ranking.STANDINGSDATE >= s15i) & (ranking.STANDINGSDATE <= s15f)] 
rk3 = ranking[(ranking.STANDINGSDATE >= s16i) & (ranking.STANDINGSDATE <= s16f)] 
rk4 = ranking[(ranking.STANDINGSDATE >= s17i) & (ranking.STANDINGSDATE <= s17f)] 
rk5 = ranking[(ranking.STANDINGSDATE >= s18i) & (ranking.STANDINGSDATE <= s18f)]
rk = pd.concat([rk1,rk2,rk3,rk4,rk5])
rk = rk.reset_index().drop(["index"], axis = 1)
rk

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612744,0,2014,2015-04-15,West,Golden State,82,67,15,0.817,39-2,28-13,NaN
1,1610612745,0,2014,2015-04-15,West,Houston,82,56,26,0.683,30-11,26-15,NaN
2,1610612746,0,2014,2015-04-15,West,L.A. Clippers,82,56,26,0.683,30-11,26-15,NaN
3,1610612763,0,2014,2015-04-15,West,Memphis,82,55,27,0.671,31-10,24-17,NaN
4,1610612759,0,2014,2015-04-15,West,San Antonio,82,55,27,0.671,33-8,22-19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29155,1610612749,0,2018,2018-09-28,East,Milwaukee,0,0,0,0.000,0-0,0-0,NaN
29156,1610612752,0,2018,2018-09-28,East,New York,0,0,0,0.000,0-0,0-0,NaN
29157,1610612753,0,2018,2018-09-28,East,Orlando,0,0,0,0.000,0-0,0-0,NaN
29158,1610612761,0,2018,2018-09-28,East,Toronto,0,0,0,0.000,0-0,0-0,NaN


In [ ]:
for i in range(0,len(datos)):
  x = datos.loc[i,['GAME_DATE_EST']].values[0]
  fecha = datetime.strptime(x, '%Y-%m-%d').date()
  f = str(fecha - timedelta(days=1))

  local = ranking[(ranking.STANDINGSDATE == f) & (ranking.TEAM_ID == datos.loc[i,['HOME_TEAM_ID']].values[0])]
  visit = ranking[(ranking.STANDINGSDATE == f) & (ranking.TEAM_ID == datos.loc[i,['VISITOR_TEAM_ID']].values[0])]

  if len(local) == 0:
    datos.loc[i, "parcial_local"] = 0
  else:
    y = local.loc[:, 'HOME_RECORD'].values[0]
    z = y.split('-')
    datos.loc[i, "parcial_local"] = int(z[0]) - int(z[1])

  if len(visit) == 0:
    datos.loc[i, "parcial_visitante"] = 0
  else:
    y = visit.loc[:, 'ROAD_RECORD'].values[0]
    z = y.split('-')
    datos.loc[i, "parcial_visitante"] = int(z[0]) - int(z[1])

  

In [ ]:
datos

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_WINS,ult7dias_local,ult7dias_visitante,win_ult5_local,win_ult5_visitante,numero_victorias_visitante,numero_victorias_local,mean_puntos_visitante,porcentaje_triples_visitante,porcentaje_tiros_libres_visitante,mean_rebotes_visitante,mean_asistencias_visitante,mean_robos_visitante,mean_tapones_visitante,mean_faltas_visitante,mean_player_visitane,mean_puntos_local,porcentaje_triples_local,porcentaje_tiros_libres_local,mean_rebotes_local,mean_asistencias_local,mean_robos_local,mean_tapones_local,mean_faltas_local,mean_player_local,conferencia_home,conferencia_visit,parcial_local,parcial_visitante
0,2019-04-10,21801222,1610612766,1610612753,0,4.0,3.0,4.0,4.0,42.0,43.0,121.8,0.393064,0.802083,47.4,32.4,8.2,5.8,18.2,11.0,111.6,0.359551,0.800000,42.6,24.0,8.8,3.8,15.6,10.0,East,East,10.0,-8.0
1,2019-04-10,21801227,1610612759,1610612742,1,3.0,4.0,3.0,3.0,34.0,50.0,118.2,0.321608,0.742188,52.0,28.4,4.2,4.6,20.4,10.0,109.8,0.351145,0.786408,43.2,24.6,7.0,5.0,17.8,12.0,West,West,22.0,-22.0
2,2019-04-10,21801223,1610612752,1610612765,0,4.0,4.0,3.0,1.0,42.0,20.0,98.4,0.305263,0.752137,42.8,19.6,7.8,5.8,22.2,10.0,103.6,0.367470,0.733333,48.0,21.6,4.0,7.8,16.6,9.0,East,East,-22.0,-12.0
3,2019-04-10,21801226,1610612749,1610612760,0,3.0,3.0,3.0,4.0,51.0,63.0,117.8,0.345550,0.736364,52.0,25.8,8.4,4.6,21.6,11.0,127.4,0.328283,0.794702,53.0,26.2,8.2,6.0,21.0,9.0,East,West,26.0,2.0
4,2019-04-10,21801220,1610612737,1610612754,0,3.0,3.0,2.0,2.0,49.0,31.0,105.6,0.384615,0.678161,41.6,27.0,6.6,4.2,17.8,11.0,119.4,0.339535,0.660870,46.2,27.4,9.6,5.2,21.2,10.0,East,East,-6.0,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6584,2014-10-06,11400007,1610612737,1610612740,1,0.0,1.0,0.0,1.0,1.0,0.0,98.0,0.437500,0.705882,42.0,19.0,6.0,6.0,25.0,15.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,East,West,0.0,1.0
6585,2014-10-06,11400006,1610612738,1610612755,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,East,East,0.0,0.0
6586,2014-10-06,11400005,1610612747,1610612743,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,West,West,0.0,0.0
6587,2014-10-05,11400002,1610612761,1610612758,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,East,West,0.0,0.0


In [ ]:
from google.colab import files

with open('datos_modelo_v3.csv', 'w') as f:
  datos.to_csv(f,  index = False)
  

files.download('datos_modelo_v3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
datos = upload_files()

Saving datos_modelo_v3.csv to datos_modelo_v3.csv
User uploaded file "datos_modelo_v3.csv" with length 1500345 bytes


In [ ]:
datos

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_WINS,ult7dias_local,ult7dias_visitante,win_ult5_local,win_ult5_visitante,numero_victorias_visitante,numero_victorias_local,mean_puntos_visitante,porcentaje_triples_visitante,porcentaje_tiros_libres_visitante,mean_rebotes_visitante,mean_asistencias_visitante,mean_robos_visitante,mean_tapones_visitante,mean_faltas_visitante,mean_player_visitane,mean_puntos_local,porcentaje_triples_local,porcentaje_tiros_libres_local,mean_rebotes_local,mean_asistencias_local,mean_robos_local,mean_tapones_local,mean_faltas_local,mean_player_local,conferencia_home,conferencia_visit,parcial_local,parcial_visitante
0,2019-04-10,21801222,1610612766,1610612753,0,4.0,3.0,4.0,4.0,42.0,43.0,121.8,0.393064,0.802083,47.4,32.4,8.2,5.8,18.2,11.0,111.6,0.359551,0.800000,42.6,24.0,8.8,3.8,15.6,10.0,East,East,10.0,-8.0
1,2019-04-10,21801227,1610612759,1610612742,1,3.0,4.0,3.0,3.0,34.0,50.0,118.2,0.321608,0.742188,52.0,28.4,4.2,4.6,20.4,10.0,109.8,0.351145,0.786408,43.2,24.6,7.0,5.0,17.8,12.0,West,West,22.0,-22.0
2,2019-04-10,21801223,1610612752,1610612765,0,4.0,4.0,3.0,1.0,42.0,20.0,98.4,0.305263,0.752137,42.8,19.6,7.8,5.8,22.2,10.0,103.6,0.367470,0.733333,48.0,21.6,4.0,7.8,16.6,9.0,East,East,-22.0,-12.0
3,2019-04-10,21801226,1610612749,1610612760,0,3.0,3.0,3.0,4.0,51.0,63.0,117.8,0.345550,0.736364,52.0,25.8,8.4,4.6,21.6,11.0,127.4,0.328283,0.794702,53.0,26.2,8.2,6.0,21.0,9.0,East,West,26.0,2.0
4,2019-04-10,21801220,1610612737,1610612754,0,3.0,3.0,2.0,2.0,49.0,31.0,105.6,0.384615,0.678161,41.6,27.0,6.6,4.2,17.8,11.0,119.4,0.339535,0.660870,46.2,27.4,9.6,5.2,21.2,10.0,East,East,-6.0,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6584,2014-10-06,11400007,1610612737,1610612740,1,0.0,1.0,0.0,1.0,1.0,0.0,98.0,0.437500,0.705882,42.0,19.0,6.0,6.0,25.0,15.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,East,West,0.0,1.0
6585,2014-10-06,11400006,1610612738,1610612755,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,East,East,0.0,0.0
6586,2014-10-06,11400005,1610612747,1610612743,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,West,West,0.0,0.0
6587,2014-10-05,11400002,1610612761,1610612758,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,East,West,0.0,0.0


In [ ]:
datos.loc[(datos.conferencia_home == "East") & (datos.conferencia_visit == "East"),'conferencia'] = 'EE' 
datos.loc[(datos.conferencia_home == "East") & (datos.conferencia_visit == "West"),'conferencia'] = 'EW' 
datos.loc[(datos.conferencia_home == "West") & (datos.conferencia_visit == "East"),'conferencia'] = 'WE' 
datos.loc[(datos.conferencia_home == "West") & (datos.conferencia_visit == "West"),'conferencia'] = 'WW' 

In [ ]:
datos

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_WINS,ult7dias_local,ult7dias_visitante,win_ult5_local,win_ult5_visitante,numero_victorias_visitante,numero_victorias_local,mean_puntos_visitante,porcentaje_triples_visitante,porcentaje_tiros_libres_visitante,mean_rebotes_visitante,mean_asistencias_visitante,mean_robos_visitante,mean_tapones_visitante,mean_faltas_visitante,mean_player_visitane,mean_puntos_local,porcentaje_triples_local,porcentaje_tiros_libres_local,mean_rebotes_local,mean_asistencias_local,mean_robos_local,mean_tapones_local,mean_faltas_local,mean_player_local,conferencia_home,conferencia_visit,parcial_local,parcial_visitante,conferencia
0,2019-04-10,21801222,1610612766,1610612753,0,4.0,3.0,4.0,4.0,42.0,43.0,121.8,0.393064,0.802083,47.4,32.4,8.2,5.8,18.2,11.0,111.6,0.359551,0.800000,42.6,24.0,8.8,3.8,15.6,10.0,East,East,10.0,-8.0,EE
1,2019-04-10,21801227,1610612759,1610612742,1,3.0,4.0,3.0,3.0,34.0,50.0,118.2,0.321608,0.742188,52.0,28.4,4.2,4.6,20.4,10.0,109.8,0.351145,0.786408,43.2,24.6,7.0,5.0,17.8,12.0,West,West,22.0,-22.0,WW
2,2019-04-10,21801223,1610612752,1610612765,0,4.0,4.0,3.0,1.0,42.0,20.0,98.4,0.305263,0.752137,42.8,19.6,7.8,5.8,22.2,10.0,103.6,0.367470,0.733333,48.0,21.6,4.0,7.8,16.6,9.0,East,East,-22.0,-12.0,EE
3,2019-04-10,21801226,1610612749,1610612760,0,3.0,3.0,3.0,4.0,51.0,63.0,117.8,0.345550,0.736364,52.0,25.8,8.4,4.6,21.6,11.0,127.4,0.328283,0.794702,53.0,26.2,8.2,6.0,21.0,9.0,East,West,26.0,2.0,EW
4,2019-04-10,21801220,1610612737,1610612754,0,3.0,3.0,2.0,2.0,49.0,31.0,105.6,0.384615,0.678161,41.6,27.0,6.6,4.2,17.8,11.0,119.4,0.339535,0.660870,46.2,27.4,9.6,5.2,21.2,10.0,East,East,-6.0,-4.0,EE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6584,2014-10-06,11400007,1610612737,1610612740,1,0.0,1.0,0.0,1.0,1.0,0.0,98.0,0.437500,0.705882,42.0,19.0,6.0,6.0,25.0,15.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,East,West,0.0,1.0,EW
6585,2014-10-06,11400006,1610612738,1610612755,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,East,East,0.0,0.0,EE
6586,2014-10-06,11400005,1610612747,1610612743,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,West,West,0.0,0.0,WW
6587,2014-10-05,11400002,1610612761,1610612758,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,East,West,0.0,0.0,EW


In [ ]:
from google.colab import files

with open('datos_modelo_v3.csv', 'w') as f:
  datos.to_csv(f,  index = False)
  

files.download('datos_modelo_v3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>